In [16]:
'''
Ref: https://discourse.holoviz.org/t/how-to-capture-the-click-event-on-plotly-plot-with-panel/1360/5
'''
from IPython.display import display
import numpy as np
import pandas as pd
import panel as pn
import param
import plotly.graph_objs as go

# set the random seed used by numpy
np.random.seed(42)

# Disable default plotly theme
import plotly.io as pio
pio.templates.default = None

pn.extension('plotly', 'tabulator')
# pn.config.sizing_mode = 'stretch_width'

# global data variable
df = pd.read_csv('../data/test_pred_glove300d.csv')

class UsefulnessRegression(param.Parameterized):
    def get_regressor_map():
        regr_algs = [x for x in df.columns if x.startswith('pred')]
        regr_alg_names = ['Random Forest Regressor', 'XGBoost Regressor', 'Kernel Ridge Regressor',
                        'Linear Regressor', 'LSTM regressor 1', 'LSTM regressor 2']
        regr_algs_names = dict(zip(regr_alg_names, regr_algs))
        return regr_algs_names
    regr_names_algs = get_regressor_map()
    regr_names = list(regr_names_algs.keys())
    game_names = sorted(list(df.title.unique()))
    regressor = param.ObjectSelector(default=regr_names[5], objects=regr_names)
    game_titles = param.ObjectSelector(default='All', objects=['All']+game_names)

    def get_selected_data(self):
        df_sel = df[['found_helpful_percentage', self.regr_names_algs[self.regressor]]].copy()
        if self.game_titles != 'All':
            df_sel = df_sel[df_sel.title == self.game_titles].copy()
        df_sel['random_score'] = np.random.uniform(low=0, high=1, size=len(df_sel))
        df_sel['mean_rgr'] = df_sel.found_helpful_percentage.mean()
        print(df_sel.columns)
        return df_sel
    
    def perf_summary(self):
        data = self.get_selected_data()
        title = 'Predicted vs. Truth Fraction of Usefulness Votes'

        # traces = []
        # for continent, df in data.groupby('continent'):
        #     marker=dict(symbol='circle', sizemode='area', sizeref=0.1, size=df['size'], line=dict(width=2))
        #     traces.append(go.Scatter(x=df.gdpPercap, y=df.lifeExp, mode='markers', marker=marker, name=continent))
        
        # axis_opts = dict(gridcolor='rgb(255, 255, 255)', zerolinewidth=1, ticklen=5, gridwidth=2)
        # layout = go.Layout(title=title, showlegend=self.show_legend, width=550,
        #     xaxis=dict(title=self.xlabel, type='log', **axis_opts),
        #     yaxis=dict( title=self.ylabel, **axis_opts))
        
        # return go.Figure(data=traces, layout=layout)
        # fig_scat = go.FigureWidget()
        # fig_scat.add_scatter(
        #     x=data.found_helpful_percentage,
        #     y=data[self.regr_names_algs[self.regressor]],
        #     mode='markers'
        # )
        traces = []
        traces.append(go.Scatter(x=data.found_helpful_percentage, y=data[self.regr_names_algs[self.regressor]], mode='markers'))
        layout = go.Layout(title=title)
        return go.Figure(data=traces, layout=layout)
    
    def error_hist(self):
        data = self.get_selected_data()
        title = 'Mean Absolute Error of the NLP Model: {:.3f} <br> Mean Absolute Error of the Random Guess: {:.3f}'.format(np.abs(data[self.regr_names_algs[self.regressor]]-data.found_helpful_percentage).mean(), np.abs(data.random_score-data.found_helpful_percentage).mean())
        fig = go.Figure(data=go.Histogram(x=(df[regr_model]-df.found_helpful_percentage), name='my model'),
                    layout=dict(title=title,
                    xaxis=dict(title='Predicted - Truth Fraction of Usefulness Votes'),
                    yaxis=dict(title='Counts')))
        fig.add_histogram(x=df.random_score-df.found_helpful_percentage, name='random guess')
        # fig.add_histogram(x=df.mean_rgr-df.found_helpful_percentage, name='trivial mean')
        fig.update_layout(barmode='overlay')
        fig.update_traces(opacity=0.5)
        return fig
    

    def scatter_plot(self):
        data = self.get_selected_data()
        x_ser = data.found_helpful_percentage
        y_ser = data[self.regr_names_algs[self.regressor]]
        x_med = x_ser.median()
        y_med = y_ser.median()
        x_min = x_ser.min()*.9
        y_min = y_ser.min()*.9
        x_max = x_ser.max()*1.1
        y_max = y_ser.max()*1.1
        fig = go.Figure(data=go.Scatter(x=x_ser, y=y_ser, mode='markers'),
                        layout=dict(title='Predicted vs. Truth Fraction of Usefulness Votes',
                        xaxis=dict(title='True Fraction of Useful Votes'),
                        yaxis=dict(title='Predicted Fraction of Useful Votes')))
        fig.add_shape(type='rect',
                    x0=x_med, y0=y_med, x1=x_max, y1=y_max, opacity=0.2, fillcolor='Green', line=dict(width=0))
        fig.add_shape(type='rect',
                    x0=x_min, y0=y_min, x1=x_med, y1=y_med, opacity=0.2, fillcolor='Green', line=dict(width=0))
        fig.add_shape(type='rect',
                    x0=x_min, y0=y_med, x1=x_med, y1=y_max, opacity=0.2, fillcolor='Red', line=dict(width=0))
        fig.add_shape(type='rect',
                    x0=x_med, y0=y_min, x1=x_max, y1=y_med, opacity=0.2, fillcolor='Red', line=dict(width=0))
        fig.layout.autosize = True    
        return fig

my_regr = UsefulnessRegression()

In [17]:
# specify the model to inspect
# regr_model = 'pred_xgbr_untuned'
regr_model = 'pred_lstm2_untuned'

# draw the performance of random guess
df['random_score'] = np.random.uniform(low=0, high=1, size=len(df))
df['mean_rgr'] = df.found_helpful_percentage.mean()

def create_plot():
    x_ser = df.found_helpful_percentage
    y_ser = df[regr_model]
    x_med = x_ser.median()
    y_med = y_ser.median()
    x_min = x_ser.min()*.9
    y_min = y_ser.min()*.9
    x_max = x_ser.max()*1.1
    y_max = y_ser.max()*1.1
    fig = go.Figure(data=go.Scatter(x=x_ser, y=y_ser, mode='markers'),
                    layout=dict(title='Predicted vs. Truth Fraction of Usefulness Votes',
                    xaxis=dict(title='True Fraction of Useful Votes'),
                    yaxis=dict(title='Predicted Fraction of Useful Votes')))
    fig.add_shape(type='rect',
                  x0=x_med, y0=y_med, x1=x_max, y1=y_max, opacity=0.2, fillcolor='Green', line=dict(width=0))
    fig.add_shape(type='rect',
                  x0=x_min, y0=y_min, x1=x_med, y1=y_med, opacity=0.2, fillcolor='Green', line=dict(width=0))
    fig.add_shape(type='rect',
                  x0=x_min, y0=y_med, x1=x_med, y1=y_max, opacity=0.2, fillcolor='Red', line=dict(width=0))
    fig.add_shape(type='rect',
                  x0=x_med, y0=y_min, x1=x_max, y1=y_med, opacity=0.2, fillcolor='Red', line=dict(width=0))
    fig.layout.autosize = True    
    return fig

def error_hist():
    ### title = 'Mean Absolute Error of the NLP Model: {:.3f} <br> Mean Absolute Error of the Random Guess: {:.3f} <br> Mean Absolute Error of the Random Guess: {:.3f}'.format(np.abs(df[regr_model]-df.found_helpful_percentage).mean(), np.abs(df.random_score-df.found_helpful_percentage).mean(), np.abs(df.mean_rgr-df.found_helpful_percentage).mean())
    title = 'Mean Absolute Error of the NLP Model: {:.3f} <br> Mean Absolute Error of the Random Guess: {:.3f}'.format(np.abs(df[regr_model]-df.found_helpful_percentage).mean(), np.abs(df.random_score-df.found_helpful_percentage).mean())
    fig = go.Figure(data=go.Histogram(x=(df[regr_model]-df.found_helpful_percentage), name='my model'),
                    layout=dict(title=title,
                    xaxis=dict(title='Predicted - Truth Fraction of Usefulness Votes'),
                    yaxis=dict(title='Counts')))
    fig.add_histogram(x=df.random_score-df.found_helpful_percentage, name='random guess')
    ### fig.add_histogram(x=df.mean_rgr-df.found_helpful_percentage, name='trivial mean')
    fig.update_layout(barmode='overlay')
    fig.update_traces(opacity=0.5)
    return fig

def error_hist2():
    title = 'Mean Absolute Error of the NLP Model: {:.3f} <br> Mean Absolute Error of the Random Guess: {:.3f} <br> Mean Absolute Error of the Random Guess: {:.3f}'.format(np.abs(df[regr_model]-df.found_helpful_percentage).mean(), np.abs(df.random_score-df.found_helpful_percentage).mean(), np.abs(df.mean_rgr-df.found_helpful_percentage).mean())
    fig = go.Figure(data=go.Histogram(x=(df[regr_model]-df.found_helpful_percentage), name='my model'),
                    layout=dict(title=title,
                    xaxis=dict(title='Predicted - Truth Fraction of Usefulness Votes'),
                    yaxis=dict(title='Counts')))
    fig.add_histogram(x=df.random_score-df.found_helpful_percentage, name='random guess')
    fig.add_histogram(x=df.mean_rgr-df.found_helpful_percentage, name='trivial mean')
    fig.update_layout(barmode='overlay')
    fig.update_traces(opacity=0.5)
    return fig

plot = create_plot()
plot2 = error_hist()
#### plot_panel = pn.pane.Plotly(plot, config={'responsive': True}, sizing_mode='stretch_both')
plot_panel = pn.pane.Plotly(plot, config={'responsive': True}, width=600, height=600)
plot_panel2 = pn.pane.Plotly(plot2, config={'responsive': True}, width=600, height=600)
plot_panel3 = pn.pane.Plotly(error_hist2(), config={'responsive': True}, width=600, height=600)

# with class
# plot_panel = pn.pane.Plotly(my_regr.scatter_plot(), config={'responsive': True}, width=600, height=600)
# plot_panel2 = pn.pane.Plotly(my_regr.error_hist(), config={'responsive': True}, width=600, height=600)

@pn.depends(plot_panel.param.click_data, watch=True)
def print_hello_world(click_data):
    print('hello world', click_data)

@pn.depends(plot_panel.param.click_data, plot_panel.param.object)
def string_hello_world(click_data, object):
    if click_data:
        df_clicked = df[(df.found_helpful_percentage == click_data['points'][0]['x'])
                        & (df[regr_model] == click_data['points'][0]['y'])][['title', 'found_helpful_percentage', regr_model]]
        print(type(object))
        # return pn.widgets.DataFrame(df_clicked.T, name='DataFrame')
        return pn.widgets.Tabulator(df_clicked.T, layout='fit_data_stretch', width=500)
    return pn.widgets.Tabulator(pd.DataFrame().T, layout='fit_data_stretch', width=500)

@pn.depends(plot_panel.param.click_data)
def show_clicked_rec(click_data):
    if click_data:
        df_clicked = df[(df.found_helpful_percentage == click_data['points'][0]['x'])
                        & (df[regr_model] == click_data['points'][0]['y'])]
        md_str = '# Clicked Review:\n\n <font size="3"> {}</font>'.format(df_clicked.iloc[0]['review'])
        # return pn.widgets.StaticText(name='Selected Review:\n', value=df_clicked.iloc[0]['review'])
        return pn.pane.Markdown(md_str, sizing_mode='stretch_width', background='#c5d86d')
    return pn.pane.Markdown('# Clicked Review:\n\n', sizing_mode='stretch_width', background='#c5d86d')


In [ ]:
desc = pn.pane.HTML("""
    The main window shows the results of trained models applied to a test dataset.""")

# sidebar widgets
widgets = pn.Param(my_regr.param, widgets={'regressor': {'type': pn.widgets.Select}, 'game': {'type': pn.widgets.Select}}, margin=0)

# decorate the page a bit...
template = pn.template.MaterialTemplate(
    title='Predicting Usefulness of Written Reviews - Click and Bring up the Individual Reviews!',
)
template.sidebar.append(desc.clone(width=250, margin=(20, 5)))
template.sidebar.append(widgets)
template.main.append(
    # pn.Row(pn.Column(plot_panel, plot_panel2), pn.Column(show_clicked_rec, string_hello_world))
    # pn.Row(plot_panel, pn.Column(show_clicked_rec, string_hello_world), plot_panel2)
    pn.Tabs(('performance summary', plot_panel2), 
            ('review inspection', pn.Row(plot_panel, pn.Column(show_clicked_rec, string_hello_world))),
            ('more info', plot_panel3)
           )
)

template.servable()